In [ ]:
# the task is to train the model through tabpfn and predict the amount for any particular productId. I want good prediction accuracy. Currently I'm getting SMAPE of 199%, go through the whole code and fix wherever required for getting good accuracy.

In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv(r"C:\Kenil Ramani\QB\GoalBot\CSV\dataset_v4.05.csv")

In [22]:
df['amount'] = df['amount'].round().astype('Int64')    # does not give error due to Int

In [23]:
df

,salesId,invoiceId,invoiceDate,stockistId,chemistId,billAmount,salesDetails_Id,productCode,productName,productId,...,contents,packingTypeId,divisionId,cityId,regionId,customerName,addressId,hqId,categoryId,chemistTypeId
0,193,DB-T-1052,7/14/2021,NaN,1758.0,124604,1418,PS35VE5K,BIOBETA-AM,166,...,Amlodipine 5 mg Tab + Metoprolol Succinate 50 ...,28,3,582.0,NaN,SAMRAT MEDICAL,69657.0,20.0,258.0,1.0
1,631,DB-T-1179,7/26/2021,NaN,1758.0,213574,4720,PS35VE5K,BIOBETA-AM,166,...,Amlodipine 5 mg Tab + Metoprolol Succinate 50 ...,28,3,582.0,NaN,SAMRAT MEDICAL,69657.0,20.0,258.0,1.0
2,304,DB-T-1084,6/29/2022,NaN,1758.0,75218,2182,PS35VE5K,BIOBETA-AM,166,...,Amlodipine 5 mg Tab + Metoprolol Succinate 50 ...,28,3,582.0,NaN,SAMRAT MEDICAL,69657.0,20.0,258.0,1.0
3,1388,DB-T-1394,7/25/2022,NaN,1758.0,151206,9994,PS35VE5K,BIOBETA-AM,166,...,Amlodipine 5 mg Tab + Metoprolol Succinate 50 ...,28,3,582.0,NaN,SAMRAT MEDICAL,69657.0,20.0,258.0,1.0
4,2543,DB-T-1726,9/14/2021,NaN,1758.0,201934,18491,PS35VE5K,BIOBETA-AM,166,...,Amlodipine 5 mg Tab + Metoprolol Succinate 50 ...,28,3,582.0,NaN,SAMRAT MEDICAL,69657.0,20.0,258.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126440,56847,DB-T-4349,1/28/2025,NaN,5469.0,13680,142754,9,BIO-ANXIT,168,...,Flupentixol 0.5 mg + Melitracen 10 mg Tab,28,3,22.0,NaN,MDC MEDI SHOP,82869.0,1.0,259.0,1.0
126441,9689,DB-T-3506,12/22/2023,NaN,5046.0,9000,70154,25,F-DIN-180,220,...,Fexofenadine 180 Mg Tab,28,1,22.0,NaN,DR.NAJABHAI RAMBHAI JALU,82446.0,34.0,NaN,NaN
126442,9689,DB-T-3506,12/22/2023,NaN,5046.0,9000,70156,12,BIOKET SOAP,215,...,Ketoconazole 2% 75 GM SOAP,35,1,22.0,NaN,DR.NAJABHAI RAMBHAI JALU,82446.0,34.0,NaN,NaN
126443,6703,DB-T-2831,11/23/2022,NaN,2425.0,1568,48809,38,R-ZOLE-DSR,235,...,Rabeprazole Sodium 20 mg Tab + Demperidone 30 ...,42,3,68.0,NaN,SHIVSAKTI MEDICAL,73053.0,9.0,259.0,1.0


In [ ]:
df['productId'].unique()                # has 138 different productIds

array([  166,    36,    46,     2,     8,    38,   177,    35,     1,
         170,    48,   180,    17,    50,   230,   216,    27,   195,
         210,   192,   248,    37,    75,   244,    79,   283,   165,
         167,   260,    71,    42,    12,    43,   228,   271,   196,
         285, 17229,   198,   206,    11,    26,   173,    70,   168,
         189,    52,    76,   217,   249,   181, 17233,   188, 17367,
       17558, 17366,   232, 17560, 17616, 17625, 17617,   220,   222,
         190,   193,   205,   325,   229,   169, 17235,   194,   233,
         223,   238,   236,   185,   255,   239,   202,   211,   209,
         201, 17244,   277,   218,   203,   213, 17234, 17249,   231,
         212,   252,   234, 17251,   241,   245, 17615,   221, 17624,
         254,   264, 17232,   226,   266,   267,   276,   251,   280,
         227, 17369, 17370,   294,   250,   253,   256,   263,   235,
         257,   278, 17553,   284,   298, 17230,   246, 17362,   261,
       17368,   270,

In [25]:
correlation = df.corr(numeric_only=True)['amount'].sort_values(ascending=False)
print(correlation)

amount                       1.000000
taxableAmount                1.000000
sgstAmount                   0.959873
cgstAmount                   0.959873
rate                         0.951413
pts                          0.951413
qty                          0.206927
billAmount                   0.150140
regionId                     0.119296
mrp                          0.070506
ptr                          0.070152
stockistId                   0.034851
packingTypeId                0.032927
free                         0.023903
categoryId                   0.017219
salesDetails_Id              0.003654
salesId                      0.003208
productId                   -0.000372
hsn                         -0.000691
cityId                      -0.003633
hqId                        -0.006781
addressId                   -0.010997
cashDiscountPercentage      -0.014361
divisionId                  -0.056654
chemistTypeId               -0.083969
chemistId                   -0.092388
sgst        

In [26]:
df_filtered = df[df['productId'] == 165].copy()

In [ ]:
# Convert to 10 bins
df_filtered['amount_class'] = pd.qcut(df_filtered['amount'], q=10, labels=False, duplicates='drop')         # i converted to 10 bins because i was getting error if not doing this

In [ ]:
features = ['taxableAmount', 'sgstAmount', 'cgstAmount', 'rate', 'pts', 'qty', 'billAmount']    # I found the relation of amount with other columns. I'm taking those columns only which has good correlation, dont change this!
X = df_filtered[features]
y = df_filtered['amount_class']

In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)        # check whether training is done properly or not, why I'm getting 199% SMAPE

In [31]:
from tabpfn import TabPFNClassifier

model = TabPFNClassifier(device="cuda")

model.fit(X_train, y_train)

TabPFNClassifier(device='cuda')

In [32]:
y_pred = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Sales')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

amount_series = df_filtered['amount']
quantile_bins = pd.qcut(amount_series, q=10, duplicates='drop', retbins=True)[1]
bin_centers = 0.5 * (quantile_bins[:-1] + quantile_bins[1:])

y_pred_class = model.predict(X_test)

y_pred = bin_centers[y_pred_class]
y_true = y_test.to_numpy()  # Ensure it's a NumPy array

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def smape(y_true, y_pred):
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    return np.mean(np.abs(y_true - y_pred) / denom) * 100

print(f"MAPE: {mape(y_true, y_pred):.2f}%")             # i'm getting inf%, please fix the issue
print(f"SMAPE: {smape(y_true, y_pred):.2f}%")           # i'm getting 199.42%, please fix the issue


MAPE: inf%
SMAPE: 199.42%


C:\Users\User1\AppData\Local\Temp\ipykernel_18252\3618198779.py:15: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
